# Extração de Sentenças e transformação para Parquet

### Este código extrai os textos do PDF e salva em parquet. Além disso, também faz a extração de sentenças onde os termos de busca aparecem.

In [1]:
import polars as pl
import PyPDF2
import os
import re
from pathlib import Path

In [2]:
def extrair_texto_pdf(caminho_pdf):
    """Extrai texto de um arquivo PDF"""
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor = PyPDF2.PdfReader(arquivo)
            texto = ""
            for pagina in leitor.pages:
                texto += pagina.extract_text() + "\n"
            return texto.strip()
    except Exception as e:
        print(f"Erro ao processar {caminho_pdf}: {e}")
        return ""

def processar_pdfs_para_parquet():
    """Processa todos os PDFs da pasta e cria um parquet"""
    pasta_origem = "STF FINAL/"
    pdfs = list(Path(pasta_origem).glob("*.pdf"))
    print(f"Encontrados {len(pdfs)} arquivos PDF")
    
    dados = []
    for i, pdf_path in enumerate(pdfs):
        print(f"Processando {i+1}/{len(pdfs)}: {pdf_path.name}")
        texto = extrair_texto_pdf(pdf_path)
        dados.append({
            "nome_pdf": pdf_path.name,
            "texto_completo": texto
        })

    df = pl.DataFrame(dados)
    df.write_parquet("documentos.parquet")
    print(f"Parquet criado com {len(dados)} documentos")

def buscar_termos_em_documentos():
    """Lê o parquet e busca termos específicos"""
    
    # Lista de termos para buscar
    termos = [
        "consequencialismo", "tragédia dos comuns", "natureza da firma", 
        "eficiência de Pareto", "produtos rivais", "produtos não-rivais", 
        "barreiras de entrada", "custo de oportunidade", "teorema de Coase", 
        "bens públicos", "perda social", "meios alternativos de solução de conflitos", 
        "análise econômica da litigância", "teoria da escolha racional", 
        "teoria dos jogos", "custos sociais", "economia comportamental", 
        "seleção adversa", "externalidades negativas", "captura regulatória", 
        "enforcement", "falhas de mercado", "excedente do consumidor", 
        "assimetria regulatória", "sistema de incentivos", "custos de transação", 
        "trade-off", "teoria da escolha pública", "assimetria de informações", 
        "problema do agente-principal", "eficiência de Kaldor–Hicks", 
        "monopólio legal", "Posner", "Coase", "Cooter", "Ulen", "Shavell", 
        "North", "Mackaay", "análise econômica do direito"
    ]
    
    def extrair_paragrafos(texto):
        """Extrai parágrafos de tamanho adequado do texto"""
        texto_limpo = re.sub(r'\n+', ' ', texto)
        texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
        frases = re.split(r'\.', texto_limpo)
        paragrafos = []
        paragrafo_atual = ""
        
        for frase in frases:
            frase = frase.strip()
            if not frase:
                continue
            
            if paragrafo_atual:
                paragrafo_atual += ". " + frase
            else:
                paragrafo_atual = frase

            if len(paragrafo_atual) >= 300 and len(paragrafo_atual) <= 800:
                paragrafos.append(paragrafo_atual + ".")
                paragrafo_atual = ""
            elif len(paragrafo_atual) > 800:
                paragrafos.append(paragrafo_atual + ".")
                paragrafo_atual = ""
                
        if paragrafo_atual:
            paragrafos.append(paragrafo_atual + ".")
        
        return paragrafos
    
    df = pl.read_parquet("documentos.parquet")
    print(f"Carregados {len(df)} documentos do parquet")
    
    resultados = []
    paragrafos_processados = set() 
    
    for linha in df.iter_rows(named=True):
        nome_pdf = linha["nome_pdf"]
        texto = linha["texto_completo"]
        
        paragrafos = extrair_paragrafos(texto)
        
        for paragrafo in paragrafos:
            termos_encontrados = []
            for termo in termos:
                if termo.lower() in paragrafo.lower():
                    termos_encontrados.append(termo)
            
            if termos_encontrados:
                chave_paragrafo = f"{nome_pdf}:{paragrafo[:100]}"
                
                if chave_paragrafo not in paragrafos_processados:
                    paragrafos_processados.add(chave_paragrafo)
                    
                    resultados.append({
                        "nome_pdf": nome_pdf,
                        "termos_encontrados": ", ".join(termos_encontrados),
                        "paragrafo_completo": paragrafo
                    })
    
    df_resultados = pl.DataFrame(resultados)
    df_resultados.write_excel("termos_encontrados.xlsx")
    
    if len(resultados) > 0:
        print(f"\nTotal de parágrafos únicos: {len(resultados)}")
        print("Primeiros 3 resultados como exemplo:")
        for i in range(min(3, len(resultados))):
            print(f"- {resultados[i]['nome_pdf']}: {resultados[i]['termos_encontrados']}")


In [ ]:
processar_pdfs_para_parquet()

Encontrados 1108 arquivos PDF
Processando 1/1108: repercussao-geral10622.pdf
Processando 2/1108: repercussao-geral10782.pdf
Processando 3/1108: repercussao-geral11202.pdf
Processando 4/1108: repercussao-geral13429.pdf
Processando 5/1108: repercussao-geral13668.pdf
Processando 6/1108: repercussao-geral13748.pdf
Processando 7/1108: repercussao-geral14152.pdf
Processando 8/1108: repercussao-geral14830.pdf
Processando 9/1108: repercussao-geral5204.pdf
Processando 10/1108: repercussao-geral5365.pdf
Processando 11/1108: repercussao-geral6341.pdf
Processando 12/1108: repercussao-geral6542.pdf
Processando 13/1108: repercussao-geral7261.pdf
Processando 14/1108: repercussao-geral9426.pdf
Processando 15/1108: sjur206020.pdf
Processando 16/1108: sjur208940.pdf
Processando 17/1108: sjur209160.pdf
Processando 18/1108: sjur210717.pdf
Processando 19/1108: sjur211402.pdf
Processando 20/1108: sjur211403.pdf
Processando 21/1108: sjur211408.pdf
Processando 22/1108: sjur212632.pdf
Processando 23/1108: sjur

In [ ]:
buscar_termos_em_documentos()

Carregados 1108 documentos do parquet

Total de parágrafos únicos: 4849
Primeiros 3 resultados como exemplo:
- repercussao-geral10622.pdf: bens públicos
- repercussao-geral10782.pdf: bens públicos
- repercussao-geral11202.pdf: Posner
